In [4]:
import pandas as pd
import numpy as np
from pathlib import Path

data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')
#data_folder = Path('E:/Downloads/')
#data_folder = Path('/sgoinfre/goinfre/Perso/pdespres/data/')

#df = pd.read_csv('E:\\Downloads\\train2_irf_0000_corr98.csv', dtype='int8')
df = pd.read_csv(data_folder/'train_redux2.csv', \
                 dtype='int8', nrows=100)
y = pd.read_csv(data_folder/'y2.csv', \
                sep=';', usecols=[1], dtype='int8', nrows=100)

print(df.shape, y.shape)

(100, 18124) (100, 1)


In [5]:
from sklearn.svm import SVC
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import StratifiedKFold
from scipy import interp

#param_range = [0.00020,0.00023,0.00025]
param_range = [1]
#gamma = 1/n
cv = StratifiedKFold(n_splits=3, random_state=42)

for i in param_range:
    
    classifier = SVC(C=i, probability=True, random_state=42)

    aucs = []
    tprs = []
    mean_fpr = np.linspace(0, 1, 100)

    for train_index, test_index in cv.split(df, y):
        model = classifier.fit(df.iloc[train_index], y.iloc[train_index].values.ravel())
        if hasattr(model, "predict_proba"):
            probas_ = model.predict_proba(df.iloc[test_index])[:, 1]
        else:  # use decision function
            probas_ = model.decision_function(df.iloc[test_index])
        fpr, tpr, thresholds = roc_curve(y.iloc[test_index], probas_)
        tprs.append(interp(mean_fpr, fpr, tpr))
        tprs[-1][0] = 0.0
        roc_auc = auc(fpr, tpr)
        aucs.append(roc_auc)

    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    print('param=%s Mean ROC (AUC = %0.3f)' % (str(i), mean_auc))
    print(aucs)

param=1 Mean ROC (AUC = 0.700)
[0.67399267399267404, 0.76984126984126988, 0.65873015873015861]


In [6]:
from sklearn.externals import joblib
joblib.dump(classifier, data_folder/'SVC.pkl') 

['/sgoinfre/goinfre/Perso/pdespres/data/SVC.pkl']